# ChE 696, 12/3/18: The modern computational science workflow

* As a class you've learned pandas, SQL, numpy, etc.
* These all come into play in the modern computational science workflow
* Computational science is the process of using computers to study specific systems; often times, we use data science methods

Today we're going to focus on three things.
1. Designing a computational experiment - 20 min
2. Analyzing/visualizing your data efficiently (timing, debugging, optimizing) - 20 min
3. Using multiprocessing to parallelize analysis - 20 min

## Preliminaries (<5 min)
If you don't already have them installed, install the time and multiprocessing packages through your favorite package manager.

In [ ]:
%matplotlib inline
import numpy as np
from math import *
import matplotlib.pyplot as plt
import time

## Computational experiment: what is the percolation threshhold in 2D? - 5 min

First: what is percolation?

![thanks wolfram](http://mathworld.wolfram.com/images/eps-gif/BondPercolation_1000.gif)

We start with a lattice, which is some ordering of sites that are connected. Here, we're going to use a simple 2D square lattice for site percolation. This is easy to work with because it is essentially an array.

Each site on the lattice (or value in the array) is then populated (given value 1) with some probability $p$. 

We call sites *clustered* if they have a neighboring occupied site. There are a number of algorithms for finding clusters; here, we're going to use something called the Hoshen-Kopelman (Kopelman is a prof at UMich!).

A *percolating cluster* exists if a cluster spans the simulation box from left to right. Spoiler alert: in 2D, this occurs at p=0.58.

Let's set up a lattice model to run our experiment.

In [ ]:
class LatticeModel(object):
    def __init__(self,L,p):
        self.L = L
        self.p = p
        self.init_lattice()
        
    def init_lattice(self):
        random_lattice = np.random.random((self.L,self.L))
        self.lattice = (random_lattice<=self.p)*1
        return

Now that we have the routine set up, let's visualize it. Here, $L$ is the side length of the simulation box. Play around with $p$.

In [ ]:
L = 20
p = 0.4

simulation = LatticeModel(L,p)
plt.imshow(simulation.lattice,cmap='binary',interpolation="none")
plt.title('Lattice value')
plt.show()

At the values I input, this is definitely not percolating. Let's walk through how we can analyze that, though.

## Analysis: Building our analysis pipeline - 25 min
Once I have a way of "simulating" my data (and the above is very simple), we need a way of analyzing it organically.

I know the below is a lot of code. One of the best things you can do to learn is read others' code (even if it's mine, which can be pretty hack-y). We're going to walk through it and look at why I set things up the way I did.

In [ ]:
class PercolationAnalysis(LatticeModel):
    def __init__(self, L, p):
        LatticeModel.__init__(self, L, p)
        self.clusterIDs = self.HK_algorithm()
    
    def init_clusters(self):
        clusterIDs = np.copy(np.zeros((self.L,self.L)))    
        for i in range(self.L):
            for j in range(self.L):
                if self.lattice[i][j]==0: pass
                else:
                    neighborIDs=[]
                    if i!=0: neighborIDs.append(clusterIDs[i-1][j])
                    if j!=0: neighborIDs.append(clusterIDs[i][j-1])
                    if neighborIDs==[]:
                        clusterIDs[i][j]=np.amax(clusterIDs)+1
                    elif max(neighborIDs)==0:
                        clusterIDs[i][j]=np.amax(clusterIDs)+1
                    else:
                        clusterIDs[i][j]=max(neighborIDs)
        return clusterIDs

    def get_neighbors(self,i,j,clusterIDs,neighborIDs):
        if i!=0:
            neighborIDs.append(clusterIDs[i-1][j])
        if j!=0:
            neighborIDs.append(clusterIDs[i][j-1])
        if i<(clusterIDs.shape[0]-1):
            neighborIDs.append(clusterIDs[i+1][j])
        if j<(clusterIDs.shape[1]-1):
            neighborIDs.append(clusterIDs[i][j+1])
        return neighborIDs

    def ID_proper(self,i,j,clusterIDs,proper_clusters,clusters0):
        neighborIDs = self.get_neighbors(i,j,clusterIDs,[])

        # 0) If site is empty, no cluster number
        if clusterIDs[i][j]==0: siteID = 0    

        # 1) If no neighbors to cluster, then keep particle in its own cluster
        elif np.amax(neighborIDs)==0:
            siteID = clusterIDs[i][j]
            np.place(proper_clusters,proper_clusters==clusterIDs[i][j],siteID)    

        # 2) If more than one neighboring site is occupied, take the smallest of the neighbor labels
        else:
            nonzero_labels = [x for x in neighborIDs if x>0]
            siteID = np.amin(np.asarray(nonzero_labels,clusterIDs[i][j]))
            np.place(proper_clusters,clusters0==clusterIDs[i][j],siteID)

            # 3) Store proper neighbor cluster values
            for x in nonzero_labels:
                np.place(proper_clusters,proper_clusters==x,siteID)

        # 4) Update cluster IDs with proper label here
        for k in range(len(clusters0)):
            np.place(clusterIDs,clusterIDs==clusters0[k],proper_clusters[k])

        return proper_clusters, clusterIDs

    def HK_algorithm(self):
        ## 1) Find unique ID for each site > "cluster labels"
        clusterIDs = self.init_clusters()

        # 2) Stores number of unique clusters in a 2-row array
        clusters0 = np.unique(clusterIDs)
        proper_clusters = np.copy(clusters0)

        # 3) Iterate through sites to determine "true" value
        for i in range(clusterIDs.shape[0]):
            for j in range(clusterIDs.shape[1]):
                proper_clusters, clusterIDs = self.ID_proper(i,j,clusterIDs,proper_clusters,clusters0)

        return clusterIDs
    
    # We'll come back to the two methods below in a little bit.
    
    def percolated_yes_or_no(self):
        clusters = np.unique(self.clusterIDs)
        # Screens out edge case where there are no empty sites
        a=(0 if clusters[0]==1 else 1)
        for c in clusters[a:]:
            check = np.copy(np.where(self.clusterIDs==c))
            if (np.amin(check[0])==0 and np.amax(check[0])==(L-1)): 
                return 1
            else: pass
        return 0
    
    def visualize_percolation_threshhold(self, reps, site_occupied_prob_range):
        percolation_prob = [] 
        for p in site_occupied_prob_range:
            tmp_data = []
            for rep in range(reps):
                tmp_data.append(PercolationAnalysis(L,p).percolated_yes_or_no())
            percolation_prob.append(np.sum(np.asarray(tmp_data))/reps)

        plt.plot(site_occupied_prob_range,percolation_prob,'bo-')
        plt.title('Probability (P) of percolating cluster existence')
        plt.ylabel('P')
        plt.xlabel('Probability (p) of a lattice site being occupied')
        plt.show()
        return

In [ ]:
# Use this block to play around and see what various code blocks do

*phew*, that was a lot of code. Typically, once I get this code working I will put it all into a separate .py file that I will load as a module to keep my notebook clean. I'm happy to demo an example if people are interested.

What's great about this? Now I can make a lattice and analyze it with one line of code.

In [ ]:
L = 20
p = 0.6

# Look how easy it is to do all that analysis in one line with the class method!
plt.imshow(PercolationAnalysis(L,p).clusterIDs,cmap='OrRd',interpolation="none")
plt.title('Cluster ID')
plt.colorbar()
plt.show()

print('Cluster IDs, for visual verification')

** Calculating the percolation threshhold **

The percolation threshhold is the critical site occupation probability above which there will be a percolating cluster. Let's calculate that, now that we have our clustering algorithms set up.

In [ ]:
L = 25
reps = 5
site_occupied_prob = np.linspace(0.0, 1.0, 20)

percolation_prob = [] 
# start = time.time()
for p in site_occupied_prob:
    tmp_data = []
    for rep in range(reps):
        tmp_data.append(PercolationAnalysis(L,p).percolated_yes_or_no())
    percolation_prob.append(np.sum(np.asarray(tmp_data))/reps)
# elapsed = time.time()-start
# print(elapsed)

plt.plot(site_occupied_prob,percolation_prob,'bo-')
plt.title('Probability (P) of percolating cluster existence')
plt.ylabel('P')
plt.xlabel('Probability (p) of a lattice site being occupied')
plt.show()

All right-- there's lots to dig into in the above.
* reps: We need statistics! We should run multiple randomized percolation configurations
* L: p=0.58 in the infinite limit; at lower L, we won't get as smooth of a result
* timing: Let's play with addings reps and L. How long does this take? How could we make it faster?
* Bonus: how would we get the actual percolation threshhold without eyeballing?

(And in case we were doing this "for real", I'd probably add this in as a class method for my analysis, below.)

In [ ]:
L = 25
reps = 3
site_occupied_prob = np.linspace(0.0, 1.0, 20)

PercolationAnalysis(L,0).visualize_percolation_threshhold(reps, site_occupied_prob)

Before we move on: if you'd like more code to read that's similar to this, [this](https://github.com/shannon-moran/computational-physics-F17/blob/master/02_HW/Moran_HW6.ipynb) example might be of interest. Generally, I highly recommend Phys 514 with Emmanuel Gull for those of you with computational science research interests. It gives you broad + useful practive with a variety of topics in computation physics, which is the basis for much of the work we do.

## Speed-ups: What is parallel processing? How can we implement it in python? - 20 min

In theory, we're all familiar with the idea of parallal processing: things take less time when we do them at the same time (in parallel) rather than waiting for the next one in line to finish (in serial).
![simple](https://www.oreilly.com/library/view/oracle-parallel-processing/156592701X/tagoreillycom20070221oreillyimages81824.png)

In practice, the speed-up is rarely this simple (as we're going to see below). However, parallelization is a powerful tool for getting some speed-up on modern computers.

**Python's multiprocessing package**

Python's multiprocessing package offers us an easy way to parallelize code. Today, we're going to do a really simple demo to show how it works.

If you're interested in this package more generally, [this](https://sebastianraschka.com/Articles/2014_multiprocessing.html) is the best tutorial I've yet come across and I recommend going throughit.

In [ ]:
import multiprocessing as mp

In [ ]:
cores = mp.cpu_count()
print(cores)

## Running functions in parallel, then gathering results

In [ ]:
def run_analysis(L,p):
    return PercolationAnalysis(L,p).percolated_yes_or_no()

def run_replicate_set(L,p_range):
    return [PercolationAnalysis(L,p).percolated_yes_or_no() for p in p_range]

In [ ]:
pool = mp.Pool(processes=cores)
results = [pool.apply_async(run_analysis, args=(L,p)) for p in np.linspace(0.0, 1.0, 20)]
output = [p.get() for p in results]

print(output)

Let's do the same thing we did above, but now in parallel.

In [ ]:
L = 25
reps = 5
site_occupied_prob = np.linspace(0.0,1.0,20)

start = time.time()

pool = mp.Pool(processes=cores)
results = [pool.apply_async(run_replicate_set, args=(L,site_occupied_prob)) for r in range(reps)]
output = [p.get() for p in results]
percolation_prob = np.asarray(output).reshape((len(output),site_occupied_prob.shape[0])).mean(axis=0)

elapsed = time.time()-start
print(elapsed)

plt.plot(site_occupied_prob,percolation_prob,'bo-')
plt.title('Probability (P) of percolating cluster existence')
plt.ylabel('P')
plt.xlabel('Probability (p) of a lattice site being occupied')
plt.show()

Let's play around with a few things.
* What's the speed-up that we see?
* What happens if we increase the number of processes? Decrease?
* Number of reps? Size of lattice?

## Running functions in parallel (each function doing their own thing)

In [ ]:
def run_visualization(L,p):
    plt.imshow(PercolationAnalysis(L,p).clusterIDs,cmap='OrRd',interpolation="none")
    plt.title('Cluster ID')
    plt.colorbar()
    plt.show()
    return

I will leave this as a homework assignment-- look into the .join(), .start() methods.